# Description

(Please, take a look at the README.md file in this directory for instructions on how to run this notebook)

This notebook compiles information about the GWAS and TWAS for a particular cohort. For example, the set of GWAS variants, variance of predicted expression of genes, etc.

It has specicfic parameters for papermill (see under `Settings` below).

This notebook is not directly run. See README.md.

# Modules

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import pickle

import numpy as np
import pandas as pd

import conf
from entity import Gene

# Settings

In [3]:
# a cohort name (it could be something like UK_BIOBANK, etc)
COHORT_NAME = None

# reference panel such as 1000G or GTEX_V8
REFERENCE_PANEL = None

# predictions models such as MASHR or ELASTIC_NET
EQTL_MODEL = None

# a string with a path pointing to an imputed GWAS
GWAS_FILE = None

# a string with a path pointing where S-PrediXcan results (tissue-specific are located
SPREDIXCAN_FOLDER = None

# an f-string with one placeholder {tissue}
SPREDIXCAN_FILE_PATTERN = None

# a string with a path pointing to an S-MultiXcan result
SMULTIXCAN_FILE = None

In [4]:
# Parameters
COHORT_NAME = "1000G_EUR"
REFERENCE_PANEL = "GTEX_V8"
EQTL_MODEL = "MASHR"
GWAS_FILE = "/project/ritchie20/projects/phenoplier/base/results/gls/null_sims/final_imputed_gwas/random.pheno0.glm-imputed.txt.gz"
SPREDIXCAN_FOLDER = (
    "/project/ritchie20/projects/phenoplier/base/results/gls/null_sims/twas/spredixcan/"
)
SPREDIXCAN_FILE_PATTERN = "random.pheno0-gtex_v8-mashr-{tissue}.csv"
SMULTIXCAN_FILE = "/project/ritchie20/projects/phenoplier/base/results/gls/null_sims/twas/smultixcan/random.pheno0-gtex_v8-mashr-smultixcan.txt"


In [5]:
assert COHORT_NAME is not None and len(COHORT_NAME) > 0, "A cohort name must be given"

COHORT_NAME = COHORT_NAME.lower()
display(f"Cohort name: {COHORT_NAME}")

'Cohort name: 1000g_eur'

In [6]:
assert (
    REFERENCE_PANEL is not None and len(REFERENCE_PANEL) > 0
), "A reference panel must be given"

display(f"Reference panel: {REFERENCE_PANEL}")

'Reference panel: GTEX_V8'

In [7]:
assert GWAS_FILE is not None and len(GWAS_FILE) > 0, "A GWAS file path must be given"
GWAS_FILE = Path(GWAS_FILE).resolve()
assert GWAS_FILE.exists(), "GWAS file does not exist"

display(f"GWAS file path: {str(GWAS_FILE)}")

'GWAS file path: /project/ritchie20/projects/phenoplier/base/results/gls/null_sims/final_imputed_gwas/random.pheno0.glm-imputed.txt.gz'

In [8]:
assert (
    SPREDIXCAN_FOLDER is not None and len(SPREDIXCAN_FOLDER) > 0
), "An S-PrediXcan folder path must be given"
SPREDIXCAN_FOLDER = Path(SPREDIXCAN_FOLDER).resolve()
assert SPREDIXCAN_FOLDER.exists(), "S-PrediXcan folder does not exist"

display(f"S-PrediXcan folder path: {str(SPREDIXCAN_FOLDER)}")

'S-PrediXcan folder path: /project/ritchie20/projects/phenoplier/base/results/gls/null_sims/twas/spredixcan'

In [9]:
assert (
    SPREDIXCAN_FILE_PATTERN is not None and len(SPREDIXCAN_FILE_PATTERN) > 0
), "An S-PrediXcan file pattern must be given"
assert (
    "{tissue}" in SPREDIXCAN_FILE_PATTERN
), "S-PrediXcan file pattern must have a '{tissue}' placeholder"

display(f"S-PrediXcan file template: {SPREDIXCAN_FILE_PATTERN}")

'S-PrediXcan file template: random.pheno0-gtex_v8-mashr-{tissue}.csv'

In [10]:
assert (
    SMULTIXCAN_FILE is not None and len(SMULTIXCAN_FILE) > 0
), "An S-MultiXcan result file path must be given"
SMULTIXCAN_FILE = Path(SMULTIXCAN_FILE).resolve()
assert SMULTIXCAN_FILE.exists(), "S-MultiXcan result file does not exist"

display(f"S-MultiXcan file path: {str(SMULTIXCAN_FILE)}")

'S-MultiXcan file path: /project/ritchie20/projects/phenoplier/base/results/gls/null_sims/twas/smultixcan/random.pheno0-gtex_v8-mashr-smultixcan.txt'

In [11]:
assert (
    EQTL_MODEL is not None and len(EQTL_MODEL) > 0
), "A prediction/eQTL model must be given"

display(f"eQTL model: {EQTL_MODEL}")

'eQTL model: MASHR'

In [12]:
OUTPUT_DIR_BASE = (
    conf.RESULTS["GLS"]
    / "gene_corrs"
    / "cohorts"
    / COHORT_NAME
    / REFERENCE_PANEL.lower()
    / EQTL_MODEL.lower()
)

OUTPUT_DIR_BASE.mkdir(parents=True, exist_ok=True)

display(f"Using output dir base: {OUTPUT_DIR_BASE}")

'Using output dir base: /project/ritchie20/projects/phenoplier/base/results/gls/gene_corrs/cohorts/1000g_eur/gtex_v8/mashr'

# Load MultiPLIER Z genes

In [13]:
multiplier_z_genes = pd.read_pickle(
    conf.MULTIPLIER["MODEL_Z_MATRIX_FILE"]
).index.tolist()

In [14]:
len(multiplier_z_genes)

6750

In [15]:
assert len(multiplier_z_genes) == len(set(multiplier_z_genes))

In [16]:
multiplier_z_genes[:5]

['GAS6', 'MMP14', 'DSP', 'MARCKSL1', 'SPARC']

# GWAS

In [17]:
gwas_file_columns = pd.read_csv(GWAS_FILE, sep="\t", nrows=2).columns
assert (
    "panel_variant_id" in gwas_file_columns
), "GWAS file must be final imputed one with column 'panel_variant_id'"
# FIXME: add other needed columns here

In [18]:
gwas_data = pd.read_csv(
    GWAS_FILE,
    sep="\t",
    usecols=["panel_variant_id", "pvalue", "zscore", "imputation_status"],
)

In [19]:
gwas_data.shape

(8325729, 4)

In [20]:
gwas_data.head()

,panel_variant_id,zscore,pvalue,imputation_status
0,chr1_54490_G_A_b38,1.567452,0.117009,original
1,chr1_87021_T_C_b38,0.244372,0.806943,imputed
2,chr1_263722_C_G_b38,1.155239,0.247993,imputed
3,chr1_594402_C_T_b38,0.851234,0.394639,imputed
4,chr1_630555_C_T_b38,0.855129,0.392480,imputed


In [21]:
gwas_data["imputation_status"].unique()

array(['original', 'imputed'], dtype=object)

In [22]:
gwas_data.dropna().shape

(8325729, 4)

In [23]:
# remove SNPs with no results
gwas_data = gwas_data.dropna()

In [24]:
gwas_data.shape

(8325729, 4)

## Save GWAS variants

In [25]:
gwas_data.head()

,panel_variant_id,zscore,pvalue,imputation_status
0,chr1_54490_G_A_b38,1.567452,0.117009,original
1,chr1_87021_T_C_b38,0.244372,0.806943,imputed
2,chr1_263722_C_G_b38,1.155239,0.247993,imputed
3,chr1_594402_C_T_b38,0.851234,0.394639,imputed
4,chr1_630555_C_T_b38,0.855129,0.392480,imputed


In [26]:
assert gwas_data["panel_variant_id"].is_unique

In [27]:
gwas_variants_ids_set = frozenset(gwas_data["panel_variant_id"])
list(gwas_variants_ids_set)[:5]

['chr1_19003051_C_T_b38',
 'chr15_49942423_A_G_b38',
 'chr7_121912537_C_T_b38',
 'chr2_11551980_A_C_b38',
 'chr9_26275642_A_C_b38']

In [28]:
with open(OUTPUT_DIR_BASE / "gwas_variant_ids.pkl", "wb") as handle:
    pickle.dump(gwas_variants_ids_set, handle, protocol=pickle.HIGHEST_PROTOCOL)

# TWAS

## Available tissues for eQTL model

In [29]:
prediction_model_tissues = conf.PHENOMEXCAN["PREDICTION_MODELS"][
    f"{EQTL_MODEL}_TISSUES"
].split(" ")

In [30]:
len(prediction_model_tissues)

49

In [31]:
prediction_model_tissues[:5]

['Whole_Blood',
 'Brain_Hypothalamus',
 'Brain_Amygdala',
 'Brain_Nucleus_accumbens_basal_ganglia',
 'Artery_Coronary']

## S-MultiXcan results

In [32]:
smultixcan_results = pd.read_csv(
    SMULTIXCAN_FILE, sep="\t", usecols=["gene", "gene_name", "pvalue", "n", "n_indep"]
)

In [33]:
smultixcan_results.shape

(22317, 5)

In [34]:
smultixcan_results = smultixcan_results.dropna()

In [35]:
smultixcan_results.shape

(22314, 5)

In [36]:
smultixcan_results = smultixcan_results.assign(
    gene_id=smultixcan_results["gene"].apply(lambda g: g.split(".")[0])
)

In [37]:
smultixcan_results.head()

,gene,gene_name,pvalue,n,n_indep,gene_id
0,ENSG00000131941.7,RHPN2,0.000040,48.0,3.0,ENSG00000131941
1,ENSG00000076650.6,GPATCH1,0.000078,40.0,3.0,ENSG00000076650
2,ENSG00000100906.10,NFKBIA,0.000096,1.0,1.0,ENSG00000100906
3,ENSG00000136319.11,TTC5,0.000109,47.0,5.0,ENSG00000136319
4,ENSG00000152990.13,ADGRA3,0.000135,41.0,12.0,ENSG00000152990


In [38]:
assert smultixcan_results["gene_id"].is_unique

### Get common genes with MultiPLIER

In [39]:
common_genes = set(multiplier_z_genes).intersection(
    set(smultixcan_results["gene_name"])
)

In [40]:
len(common_genes)

6444

In [41]:
sorted(list(common_genes))[:5]

['A2M', 'AAAS', 'AANAT', 'AARS', 'AARS2']

## Genes info

In [42]:
multiplier_gene_obj = {
    gene_name: Gene(name=gene_name)
    for gene_name in common_genes
    if gene_name in Gene.GENE_NAME_TO_ID_MAP
}

In [43]:
# delete common_genes, from now on, genes_info should be used for common genes
del common_genes

In [44]:
len(multiplier_gene_obj)

6444

In [45]:
assert multiplier_gene_obj["GAS6"].ensembl_id == "ENSG00000183087"

In [46]:
_gene_obj = list(multiplier_gene_obj.values())

genes_info = pd.DataFrame(
    {
        "name": [g.name for g in _gene_obj],
        "id": [g.ensembl_id for g in _gene_obj],
        "chr": [g.chromosome for g in _gene_obj],
        "band": [g.band for g in _gene_obj],
        "start_position": [g.get_attribute("start_position") for g in _gene_obj],
        "end_position": [g.get_attribute("end_position") for g in _gene_obj],
    }
)

In [47]:
genes_info = genes_info.assign(
    gene_length=genes_info.apply(
        lambda x: x["end_position"] - x["start_position"], axis=1
    )
)

In [48]:
assert genes_info["name"].is_unique

In [49]:
assert genes_info["id"].is_unique

In [50]:
genes_info.shape

(6444, 7)

In [51]:
genes_info.head()

,name,id,chr,band,start_position,end_position,gene_length
0,HEYL,ENSG00000163909,1,1p34.2,39623435.0,39639643.0,16208.0
1,CYP24A1,ENSG00000019186,20,20q13.2,54153446.0,54173986.0,20540.0
2,ITGB4,ENSG00000132470,17,17q25.1,75721328.0,75757818.0,36490.0
3,SPINT1,ENSG00000166145,15,15q15.1,40844018.0,40858207.0,14189.0
4,PCSK6,ENSG00000140479,15,15q26.3,101297142.0,101525202.0,228060.0


In [52]:
genes_info.sort_values("chr")

,name,id,chr,band,start_position,end_position,gene_length
0,HEYL,ENSG00000163909,1,1p34.2,39623435.0,39639643.0,16208.0
2190,KCNH1,ENSG00000143473,1,1q32.2,210676823.0,211134165.0,457342.0
823,LRRC41,ENSG00000132128,1,1p33,46261196.0,46303608.0,42412.0
3926,NTRK1,ENSG00000198400,1,1q23.1,156815640.0,156881850.0,66210.0
3928,PMVK,ENSG00000163344,1,1q21.3,154924740.0,154936719.0,11979.0
...,...,...,...,...,...,...,...
3552,SLC27A4,ENSG00000167114,9,9q34.11,128340527.0,128361470.0,20943.0
4546,ORM1,ENSG00000229314,9,9q32,114323056.0,114326475.0,3419.0
312,FBXW5,ENSG00000159069,9,9q34.3,136940435.0,136944738.0,4303.0
1223,TMEM133,ENSG00000170647,None,None,NaN,NaN,NaN


### Save

In [53]:
genes_info.to_pickle(OUTPUT_DIR_BASE / "genes_info.pkl")

## S-PrediXcan results

### Load results across all tissues

In [54]:
spredixcan_result_files = {
    t: SPREDIXCAN_FOLDER / SPREDIXCAN_FILE_PATTERN.format(tissue=t)
    for t in prediction_model_tissues
}

In [55]:
assert len(spredixcan_result_files) == len(prediction_model_tissues)
display(list(spredixcan_result_files.values())[:5])

[PosixPath('/project/ritchie20/projects/phenoplier/base/results/gls/null_sims/twas/spredixcan/random.pheno0-gtex_v8-mashr-Whole_Blood.csv'),
 PosixPath('/project/ritchie20/projects/phenoplier/base/results/gls/null_sims/twas/spredixcan/random.pheno0-gtex_v8-mashr-Brain_Hypothalamus.csv'),
 PosixPath('/project/ritchie20/projects/phenoplier/base/results/gls/null_sims/twas/spredixcan/random.pheno0-gtex_v8-mashr-Brain_Amygdala.csv'),
 PosixPath('/project/ritchie20/projects/phenoplier/base/results/gls/null_sims/twas/spredixcan/random.pheno0-gtex_v8-mashr-Brain_Nucleus_accumbens_basal_ganglia.csv'),
 PosixPath('/project/ritchie20/projects/phenoplier/base/results/gls/null_sims/twas/spredixcan/random.pheno0-gtex_v8-mashr-Artery_Coronary.csv')]

In [56]:
# look at the structure of one result
pd.read_csv(spredixcan_result_files["Whole_Blood"]).head()

,gene,gene_name,zscore,effect_size,pvalue,var_g,pred_perf_r2,pred_perf_pval,pred_perf_qval,n_snps_used,n_snps_in_cov,n_snps_in_model,best_gwas_p,largest_weight
0,ENSG00000131236.16,CAP1,-3.882684,NaN,0.000103,8.369900e-05,NaN,NaN,NaN,1,1,1,0.000103,0.021593
1,ENSG00000130787.13,HIP1R,3.862583,NaN,0.000112,3.039669e-03,NaN,NaN,NaN,2,2,2,0.001681,0.087495
2,ENSG00000103018.16,CYB5B,3.478682,NaN,0.000504,6.455623e-03,NaN,NaN,NaN,1,1,1,0.000504,0.114291
3,ENSG00000117906.13,RCN2,3.431258,NaN,0.000601,5.848182e-07,NaN,NaN,NaN,1,1,1,0.000601,0.001696
4,ENSG00000135469.13,COQ10A,3.363631,NaN,0.000769,4.312235e-04,NaN,NaN,NaN,2,2,2,0.013565,0.064460


In [57]:
assert all(f.exists() for f in spredixcan_result_files.values())

In [58]:
spredixcan_dfs = [
    pd.read_csv(
        f,
        usecols=[
            "gene",
            "zscore",
            "pvalue",
            "n_snps_used",
            "n_snps_in_model",
        ],
    )
    .dropna(subset=["gene", "zscore", "pvalue"])
    .assign(tissue=t)
    for t, f in spredixcan_result_files.items()
]

In [59]:
assert len(spredixcan_dfs) == len(prediction_model_tissues)

In [60]:
spredixcan_dfs = pd.concat(spredixcan_dfs)

In [61]:
assert spredixcan_dfs["tissue"].unique().shape[0] == len(prediction_model_tissues)

In [62]:
spredixcan_dfs.shape

(652858, 6)

In [63]:
spredixcan_dfs = spredixcan_dfs.assign(
    gene_id=spredixcan_dfs["gene"].apply(lambda g: g.split(".")[0])
)

In [64]:
spredixcan_dfs.head()

,gene,zscore,pvalue,n_snps_used,n_snps_in_model,tissue,gene_id
0,ENSG00000131236.16,-3.882684,0.000103,1,1,Whole_Blood,ENSG00000131236
1,ENSG00000130787.13,3.862583,0.000112,2,2,Whole_Blood,ENSG00000130787
2,ENSG00000103018.16,3.478682,0.000504,1,1,Whole_Blood,ENSG00000103018
3,ENSG00000117906.13,3.431258,0.000601,1,1,Whole_Blood,ENSG00000117906
4,ENSG00000135469.13,3.363631,0.000769,2,2,Whole_Blood,ENSG00000135469


In [65]:
# leave only common genes
spredixcan_dfs = spredixcan_dfs[spredixcan_dfs["gene_id"].isin(set(genes_info["id"]))]

In [66]:
spredixcan_dfs.shape

(233428, 7)

### Count number of tissues available per gene

In [67]:
spredixcan_genes_n_models = spredixcan_dfs.groupby("gene_id")["tissue"].nunique()

In [68]:
spredixcan_genes_n_models

gene_id
ENSG00000000419     2
ENSG00000000938    36
ENSG00000000971    34
ENSG00000001084    32
ENSG00000001167    40
                   ..
ENSG00000278540    36
ENSG00000278828     4
ENSG00000278845    49
ENSG00000281005    49
ENSG00000282608    36
Name: tissue, Length: 6444, dtype: int64

In [69]:
# testing that in S-MultiXcan I get the same number of tissues per gene
_tmp_smultixcan_results_n_models = (
    smultixcan_results.set_index("gene_id")["n"].astype(int).rename("tissue")
)

_cg = _tmp_smultixcan_results_n_models.index.intersection(
    spredixcan_genes_n_models.index
)
_tmp_smultixcan_results_n_models = _tmp_smultixcan_results_n_models.loc[_cg]
_spredixcan = spredixcan_genes_n_models.loc[_cg]

assert _spredixcan.shape[0] == _tmp_smultixcan_results_n_models.shape[0]
assert _spredixcan.equals(_tmp_smultixcan_results_n_models.loc[_spredixcan.index])

### Get tissues available per gene

In [70]:
spredixcan_genes_models = spredixcan_dfs.groupby("gene_id")["tissue"].apply(
    lambda x: frozenset(x.tolist())
)

In [71]:
spredixcan_genes_models

gene_id
ENSG00000000419         (Brain_Hypothalamus, Brain_Substantia_nigra)
ENSG00000000938    (Esophagus_Muscularis, Thyroid, Brain_Cortex, ...
ENSG00000000971    (Esophagus_Muscularis, Thyroid, Brain_Cortex, ...
ENSG00000001084    (Thyroid, Brain_Cortex, Heart_Left_Ventricle, ...
ENSG00000001167    (Esophagus_Muscularis, Thyroid, Brain_Cortex, ...
                                         ...                        
ENSG00000278540    (Esophagus_Muscularis, Thyroid, Brain_Cortex, ...
ENSG00000278828    (Cells_EBV-transformed_lymphocytes, Adipose_Vi...
ENSG00000278845    (Esophagus_Muscularis, Thyroid, Brain_Cortex, ...
ENSG00000281005    (Esophagus_Muscularis, Thyroid, Brain_Cortex, ...
ENSG00000282608    (Esophagus_Muscularis, Brain_Cortex, Heart_Lef...
Name: tissue, Length: 6444, dtype: object

In [72]:
assert spredixcan_genes_n_models.shape[0] == spredixcan_genes_models.shape[0]

In [73]:
assert spredixcan_genes_n_models.index.equals(spredixcan_genes_models.index)

In [74]:
assert (spredixcan_genes_models.apply(len) <= len(prediction_model_tissues)).all()

In [75]:
spredixcan_genes_models.apply(len).describe()

count    6444.000000
mean       36.224084
std        12.974649
min         1.000000
25%        29.000000
50%        41.000000
75%        47.000000
max        49.000000
Name: tissue, dtype: float64

In [76]:
# testing
assert (
    spredixcan_genes_models.loc[spredixcan_genes_n_models.index]
    .apply(len)
    .equals(spredixcan_genes_n_models)
)

### Add gene name and set index

In [77]:
spredixcan_genes_models = spredixcan_genes_models.to_frame().reset_index()

In [78]:
spredixcan_genes_models.head()

,gene_id,tissue
0,ENSG00000000419,"(Brain_Hypothalamus, Brain_Substantia_nigra)"
1,ENSG00000000938,"(Esophagus_Muscularis, Thyroid, Brain_Cortex, ..."
2,ENSG00000000971,"(Esophagus_Muscularis, Thyroid, Brain_Cortex, ..."
3,ENSG00000001084,"(Thyroid, Brain_Cortex, Heart_Left_Ventricle, ..."
4,ENSG00000001167,"(Esophagus_Muscularis, Thyroid, Brain_Cortex, ..."


In [79]:
spredixcan_genes_models = spredixcan_genes_models.assign(
    gene_name=spredixcan_genes_models["gene_id"].apply(
        lambda g: Gene.GENE_ID_TO_NAME_MAP[g]
    )
)

In [80]:
spredixcan_genes_models = spredixcan_genes_models[["gene_id", "gene_name", "tissue"]]

In [81]:
spredixcan_genes_models = spredixcan_genes_models.set_index("gene_id")

In [82]:
spredixcan_genes_models.head()

,gene_name,tissue
gene_id,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra)"
ENSG00000000938,FGR,"(Esophagus_Muscularis, Thyroid, Brain_Cortex, ..."
ENSG00000000971,CFH,"(Esophagus_Muscularis, Thyroid, Brain_Cortex, ..."
ENSG00000001084,GCLC,"(Thyroid, Brain_Cortex, Heart_Left_Ventricle, ..."
ENSG00000001167,NFYA,"(Esophagus_Muscularis, Thyroid, Brain_Cortex, ..."


### Add number of tissues

In [83]:
spredixcan_genes_models = spredixcan_genes_models.assign(
    n_tissues=spredixcan_genes_models["tissue"].apply(len)
)

In [84]:
spredixcan_genes_models.head()

,gene_name,tissue,n_tissues
gene_id,,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra)",2
ENSG00000000938,FGR,"(Esophagus_Muscularis, Thyroid, Brain_Cortex, ...",36
ENSG00000000971,CFH,"(Esophagus_Muscularis, Thyroid, Brain_Cortex, ...",34
ENSG00000001084,GCLC,"(Thyroid, Brain_Cortex, Heart_Left_Ventricle, ...",32
ENSG00000001167,NFYA,"(Esophagus_Muscularis, Thyroid, Brain_Cortex, ...",40


### Get gene's objects

In [85]:
spredixcan_gene_obj = {
    gene_id: Gene(ensembl_id=gene_id) for gene_id in spredixcan_genes_models.index
}

In [86]:
len(spredixcan_gene_obj)

6444

### Add genes' variance captured by principal components

In [87]:
def _get_gene_pc_variance(gene_row):
    gene_id = gene_row.name
    gene_tissues = gene_row["tissue"]
    gene_obj = spredixcan_gene_obj[gene_id]

    u, s, vt = gene_obj.get_tissues_correlations_svd(
        tissues=gene_tissues,
        snps_subset=gwas_variants_ids_set,
        reference_panel=REFERENCE_PANEL,
        model_type=EQTL_MODEL,
        # use_covariance_matrix=True,
    )

    return s

In [88]:
_tmp = spredixcan_genes_models.loc["ENSG00000188976"]
_get_gene_pc_variance(_tmp)

array([35.14301823,  3.87568728,  2.3091285 ,  1.90683623,  1.27573165])

In [89]:
spredixcan_genes_tissues_pc_variance = spredixcan_genes_models.apply(
    _get_gene_pc_variance, axis=1
)

In [90]:
spredixcan_genes_tissues_pc_variance

gene_id
ENSG00000000419             [1.0442398458252409, 0.9557601541747592]
ENSG00000000938    [31.63266933637129, 2.078367751851144, 1.27161...
ENSG00000000971    [21.56076992568329, 7.310170987045463, 1.83778...
ENSG00000001084    [21.291939080546538, 4.699418748743195, 2.2489...
ENSG00000001167               [37.3356602954977, 1.3914019604538357]
                                         ...                        
ENSG00000278540    [30.239540272954756, 3.0448326465798896, 1.679...
ENSG00000278828              [3.062182335109643, 0.9378176648903567]
ENSG00000278845              [45.687454510139084, 2.226801954648235]
ENSG00000281005                                  [48.30170739785572]
ENSG00000282608    [22.6535140812371, 6.245520030604538, 2.555945...
Length: 6444, dtype: object

In [91]:
# # testing
# assert spredixcan_genes_tissues_pc_variance.loc[
#     "ENSG00000188976"
# ].sum() == pytest.approx(44.01605629086847)
# # this is using the covariance:
# # assert spredixcan_genes_tissues_pc_variance.loc["ENSG00000188976"].sum() == pytest.approx(1.1492946006449425)

In [92]:
# add to spredixcan_genes_models
spredixcan_genes_models = spredixcan_genes_models.join(
    spredixcan_genes_tissues_pc_variance.rename("tissues_pc_variances")
)

In [93]:
spredixcan_genes_models.shape

(6444, 4)

In [94]:
spredixcan_genes_models.head()

,gene_name,tissue,n_tissues,tissues_pc_variances
gene_id,,,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra)",2,"[1.0442398458252409, 0.9557601541747592]"
ENSG00000000938,FGR,"(Esophagus_Muscularis, Thyroid, Brain_Cortex, ...",36,"[31.63266933637129, 2.078367751851144, 1.27161..."
ENSG00000000971,CFH,"(Esophagus_Muscularis, Thyroid, Brain_Cortex, ...",34,"[21.56076992568329, 7.310170987045463, 1.83778..."
ENSG00000001084,GCLC,"(Thyroid, Brain_Cortex, Heart_Left_Ventricle, ...",32,"[21.291939080546538, 4.699418748743195, 2.2489..."
ENSG00000001167,NFYA,"(Esophagus_Muscularis, Thyroid, Brain_Cortex, ...",40,"[37.3356602954977, 1.3914019604538357]"


### Add genes' variance captured by principal components (covariance)

In [95]:
def _get_gene_pc_variance(gene_row):
    gene_id = gene_row.name
    gene_tissues = gene_row["tissue"]
    gene_obj = spredixcan_gene_obj[gene_id]

    u, s, vt = gene_obj.get_tissues_correlations_svd(
        tissues=gene_tissues,
        snps_subset=gwas_variants_ids_set,
        reference_panel=REFERENCE_PANEL,
        model_type=EQTL_MODEL,
        use_covariance_matrix=True,
    )

    return s

In [96]:
_tmp = spredixcan_genes_models.loc["ENSG00000188976"]
_get_gene_pc_variance(_tmp)

array([0.92569322, 0.08878404])

In [97]:
spredixcan_genes_tissues_pc_variance = spredixcan_genes_models.apply(
    _get_gene_pc_variance, axis=1
)

In [98]:
spredixcan_genes_tissues_pc_variance

gene_id
ENSG00000000419       [0.013163142634155849, 0.00048574890347684763]
ENSG00000000938                                 [0.1715948657078027]
ENSG00000000971    [0.2093060319587451, 0.058234347656008024, 0.0...
ENSG00000001084    [0.39430527375123775, 0.11627488888534836, 0.0...
ENSG00000001167            [1.1642639979821048, 0.05742881287671712]
                                         ...                        
ENSG00000278540          [0.12105177321833696, 0.011043537807199575]
ENSG00000278828        [0.003774008788117112, 0.0024289076167174424]
ENSG00000278845            [1.0913874047417143, 0.04698044450101883]
ENSG00000281005                                 [3.8662049265278173]
ENSG00000282608    [0.22315950079956806, 0.031779000782788086, 0....
Length: 6444, dtype: object

In [99]:
# # testing
# # assert spredixcan_genes_tissues_pc_variance.loc["ENSG00000188976"].sum() == pytest.approx(44.01605629086847)
# # this is using the covariance:
# assert spredixcan_genes_tissues_pc_variance.loc[
#     "ENSG00000188976"
# ].sum() == pytest.approx(1.1492946006449425)

In [100]:
# add to spredixcan_genes_models
spredixcan_genes_models = spredixcan_genes_models.join(
    spredixcan_genes_tissues_pc_variance.rename("tissues_pc_variances_cov")
)

In [101]:
spredixcan_genes_models.shape

(6444, 5)

In [102]:
spredixcan_genes_models.head()

,gene_name,tissue,n_tissues,tissues_pc_variances,tissues_pc_variances_cov
gene_id,,,,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra)",2,"[1.0442398458252409, 0.9557601541747592]","[0.013163142634155849, 0.00048574890347684763]"
ENSG00000000938,FGR,"(Esophagus_Muscularis, Thyroid, Brain_Cortex, ...",36,"[31.63266933637129, 2.078367751851144, 1.27161...",[0.1715948657078027]
ENSG00000000971,CFH,"(Esophagus_Muscularis, Thyroid, Brain_Cortex, ...",34,"[21.56076992568329, 7.310170987045463, 1.83778...","[0.2093060319587451, 0.058234347656008024, 0.0..."
ENSG00000001084,GCLC,"(Thyroid, Brain_Cortex, Heart_Left_Ventricle, ...",32,"[21.291939080546538, 4.699418748743195, 2.2489...","[0.39430527375123775, 0.11627488888534836, 0.0..."
ENSG00000001167,NFYA,"(Esophagus_Muscularis, Thyroid, Brain_Cortex, ...",40,"[37.3356602954977, 1.3914019604538357]","[1.1642639979821048, 0.05742881287671712]"


### Add gene variance per tissue

In [103]:
def _get_gene_variances(gene_row):
    gene_id = gene_row.name
    gene_tissues = gene_row["tissue"]

    tissue_variances = {}
    gene_obj = spredixcan_gene_obj[gene_id]

    for tissue in gene_tissues:
        tissue_var = gene_obj.get_pred_expression_variance(
            tissue=tissue,
            reference_panel=REFERENCE_PANEL,
            model_type=EQTL_MODEL,
            snps_subset=gwas_variants_ids_set,
        )

        if tissue_var is not None:
            tissue_variances[tissue] = tissue_var

    return tissue_variances

In [104]:
_tmp = spredixcan_genes_models.loc["ENSG00000000419"]
_get_gene_variances(_tmp)

{'Brain_Hypothalamus': 0.013162153504206677,
 'Brain_Substantia_nigra': 0.0004867380334260178}

In [105]:
spredixcan_genes_tissues_variance = spredixcan_genes_models.apply(
    _get_gene_variances, axis=1
)

In [106]:
spredixcan_genes_tissues_variance

gene_id
ENSG00000000419    {'Brain_Hypothalamus': 0.013162153504206677, '...
ENSG00000000938    {'Esophagus_Muscularis': 0.006536602044478473,...
ENSG00000000971    {'Esophagus_Muscularis': 0.01739410990047228, ...
ENSG00000001084    {'Thyroid': 0.009654918580686024, 'Brain_Corte...
ENSG00000001167    {'Esophagus_Muscularis': 0.0318781600881541, '...
                                         ...                        
ENSG00000278540    {'Esophagus_Muscularis': 0.002903665335021846,...
ENSG00000278828    {'Cells_EBV-transformed_lymphocytes': 0.003027...
ENSG00000278845    {'Esophagus_Muscularis': 0.018986238666239108,...
ENSG00000281005    {'Esophagus_Muscularis': 0.011785380242520696,...
ENSG00000282608    {'Esophagus_Muscularis': 0.0073249064101135325...
Length: 6444, dtype: object

In [107]:
# # testing
# _gene_id = "ENSG00000188976"
# x = spredixcan_genes_tissues_variance.loc[_gene_id]
# # expected value obtained by sum of PCA eigenvalues on this gene's predicted expression
# assert np.sum(list(x.values())) == pytest.approx(1.2326202607409493)

In [108]:
# testing
spredixcan_genes_tissues_variance.loc["ENSG00000000419"]

{'Brain_Hypothalamus': 0.013162153504206677,
 'Brain_Substantia_nigra': 0.0004867380334260178}

In [109]:
# FIXME: maybe add more tests, these differt from GTEX V8
# # testing
# # here values were obtained from S-PrediXcan results, where the reference panel is GTEX V8, not 1000G, so just approximations)
# _gene_id = "ENSG00000000419"
# assert spredixcan_genes_tissues_variance.loc[_gene_id]["Brain_Substantia_nigra"] == pytest.approx(0.0004266255268163448)
# assert spredixcan_genes_tissues_variance.loc[_gene_id]["Brain_Hypothalamus"] == pytest.approx(0.011235877515236132)

In [110]:
# add to spredixcan_genes_models
spredixcan_genes_models = spredixcan_genes_models.join(
    spredixcan_genes_tissues_variance.rename("tissues_variances")
)

In [111]:
spredixcan_genes_models.shape

(6444, 6)

In [112]:
spredixcan_genes_models.head()

,gene_name,tissue,n_tissues,tissues_pc_variances,tissues_pc_variances_cov,tissues_variances
gene_id,,,,,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra)",2,"[1.0442398458252409, 0.9557601541747592]","[0.013163142634155849, 0.00048574890347684763]","{'Brain_Hypothalamus': 0.013162153504206677, '..."
ENSG00000000938,FGR,"(Esophagus_Muscularis, Thyroid, Brain_Cortex, ...",36,"[31.63266933637129, 2.078367751851144, 1.27161...",[0.1715948657078027],"{'Esophagus_Muscularis': 0.006536602044478473,..."
ENSG00000000971,CFH,"(Esophagus_Muscularis, Thyroid, Brain_Cortex, ...",34,"[21.56076992568329, 7.310170987045463, 1.83778...","[0.2093060319587451, 0.058234347656008024, 0.0...","{'Esophagus_Muscularis': 0.01739410990047228, ..."
ENSG00000001084,GCLC,"(Thyroid, Brain_Cortex, Heart_Left_Ventricle, ...",32,"[21.291939080546538, 4.699418748743195, 2.2489...","[0.39430527375123775, 0.11627488888534836, 0.0...","{'Thyroid': 0.009654918580686024, 'Brain_Corte..."
ENSG00000001167,NFYA,"(Esophagus_Muscularis, Thyroid, Brain_Cortex, ...",40,"[37.3356602954977, 1.3914019604538357]","[1.1642639979821048, 0.05742881287671712]","{'Esophagus_Muscularis': 0.0318781600881541, '..."


### Count number of SNPs predictors used across tissue models

In [113]:
spredixcan_genes_sum_of_n_snps_used = (
    spredixcan_dfs.groupby("gene_id")["n_snps_used"].sum().rename("n_snps_used_sum")
)

In [114]:
spredixcan_genes_sum_of_n_snps_used

gene_id
ENSG00000000419     2
ENSG00000000938    40
ENSG00000000971    44
ENSG00000001084    46
ENSG00000001167    47
                   ..
ENSG00000278540    44
ENSG00000278828     5
ENSG00000278845    89
ENSG00000281005    81
ENSG00000282608    40
Name: n_snps_used_sum, Length: 6444, dtype: int64

In [115]:
# add sum of snps used to spredixcan_genes_models
spredixcan_genes_models = spredixcan_genes_models.join(
    spredixcan_genes_sum_of_n_snps_used
)

In [116]:
spredixcan_genes_models.shape

(6444, 7)

In [117]:
spredixcan_genes_models.head()

,gene_name,tissue,n_tissues,tissues_pc_variances,tissues_pc_variances_cov,tissues_variances,n_snps_used_sum
gene_id,,,,,,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra)",2,"[1.0442398458252409, 0.9557601541747592]","[0.013163142634155849, 0.00048574890347684763]","{'Brain_Hypothalamus': 0.013162153504206677, '...",2
ENSG00000000938,FGR,"(Esophagus_Muscularis, Thyroid, Brain_Cortex, ...",36,"[31.63266933637129, 2.078367751851144, 1.27161...",[0.1715948657078027],"{'Esophagus_Muscularis': 0.006536602044478473,...",40
ENSG00000000971,CFH,"(Esophagus_Muscularis, Thyroid, Brain_Cortex, ...",34,"[21.56076992568329, 7.310170987045463, 1.83778...","[0.2093060319587451, 0.058234347656008024, 0.0...","{'Esophagus_Muscularis': 0.01739410990047228, ...",44
ENSG00000001084,GCLC,"(Thyroid, Brain_Cortex, Heart_Left_Ventricle, ...",32,"[21.291939080546538, 4.699418748743195, 2.2489...","[0.39430527375123775, 0.11627488888534836, 0.0...","{'Thyroid': 0.009654918580686024, 'Brain_Corte...",46
ENSG00000001167,NFYA,"(Esophagus_Muscularis, Thyroid, Brain_Cortex, ...",40,"[37.3356602954977, 1.3914019604538357]","[1.1642639979821048, 0.05742881287671712]","{'Esophagus_Muscularis': 0.0318781600881541, '...",47


### Count number of SNPs predictors in models across tissue models

In [118]:
spredixcan_genes_sum_of_n_snps_in_model = (
    spredixcan_dfs.groupby("gene_id")["n_snps_in_model"]
    .sum()
    .rename("n_snps_in_model_sum")
)

In [119]:
spredixcan_genes_sum_of_n_snps_in_model

gene_id
ENSG00000000419     2
ENSG00000000938    40
ENSG00000000971    44
ENSG00000001084    46
ENSG00000001167    48
                   ..
ENSG00000278540    44
ENSG00000278828     5
ENSG00000278845    91
ENSG00000281005    81
ENSG00000282608    40
Name: n_snps_in_model_sum, Length: 6444, dtype: int64

In [120]:
# add sum of snps in model to spredixcan_genes_models
spredixcan_genes_models = spredixcan_genes_models.join(
    spredixcan_genes_sum_of_n_snps_in_model
)

In [121]:
spredixcan_genes_models.shape

(6444, 8)

In [122]:
spredixcan_genes_models.head()

,gene_name,tissue,n_tissues,tissues_pc_variances,tissues_pc_variances_cov,tissues_variances,n_snps_used_sum,n_snps_in_model_sum
gene_id,,,,,,,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra)",2,"[1.0442398458252409, 0.9557601541747592]","[0.013163142634155849, 0.00048574890347684763]","{'Brain_Hypothalamus': 0.013162153504206677, '...",2,2
ENSG00000000938,FGR,"(Esophagus_Muscularis, Thyroid, Brain_Cortex, ...",36,"[31.63266933637129, 2.078367751851144, 1.27161...",[0.1715948657078027],"{'Esophagus_Muscularis': 0.006536602044478473,...",40,40
ENSG00000000971,CFH,"(Esophagus_Muscularis, Thyroid, Brain_Cortex, ...",34,"[21.56076992568329, 7.310170987045463, 1.83778...","[0.2093060319587451, 0.058234347656008024, 0.0...","{'Esophagus_Muscularis': 0.01739410990047228, ...",44,44
ENSG00000001084,GCLC,"(Thyroid, Brain_Cortex, Heart_Left_Ventricle, ...",32,"[21.291939080546538, 4.699418748743195, 2.2489...","[0.39430527375123775, 0.11627488888534836, 0.0...","{'Thyroid': 0.009654918580686024, 'Brain_Corte...",46,46
ENSG00000001167,NFYA,"(Esophagus_Muscularis, Thyroid, Brain_Cortex, ...",40,"[37.3356602954977, 1.3914019604538357]","[1.1642639979821048, 0.05742881287671712]","{'Esophagus_Muscularis': 0.0318781600881541, '...",47,48


### Summarize prediction models for each gene

In [123]:
def _summarize_gene_models(gene_id):
    """
    For a given gene ID, it returns a dataframe with predictor SNPs in rows and tissues in columns, where
    values are the weights of SNPs in those tissues.
    It can contain NaNs.
    """
    gene_obj = spredixcan_gene_obj[gene_id]
    gene_tissues = spredixcan_genes_models.loc[gene_id, "tissue"]

    gene_models = {}
    gene_unique_snps = set()
    for t in gene_tissues:
        gene_model = gene_obj.get_prediction_weights(tissue=t, model_type=EQTL_MODEL)
        gene_models[t] = gene_model

        gene_unique_snps.update(set(gene_model.index))

    df = pd.DataFrame(
        data=np.nan, index=list(gene_unique_snps), columns=list(gene_tissues)
    )

    for t in df.columns:
        for snp in df.index:
            gene_model = gene_models[t]

            if snp in gene_model.index:
                df.loc[snp, t] = gene_model.loc[snp]

    return df

In [124]:
# testing
spredixcan_gene_obj["ENSG00000000419"].get_prediction_weights(
    tissue="Brain_Hypothalamus", model_type=EQTL_MODEL
)

varID
chr20_50862947_C_T_b38    0.431375
Name: weight, dtype: float64

In [125]:
spredixcan_gene_obj["ENSG00000000419"].get_prediction_weights(
    tissue="Brain_Substantia_nigra", model_type=EQTL_MODEL
)

varID
chr20_50957480_C_T_b38   -0.146796
Name: weight, dtype: float64

In [126]:
# # testing
# _gene_id = "ENSG00000000419"

# _gene_model = _summarize_gene_models(_gene_id)
# assert (
#     _gene_model.loc["chr20_50862947_C_T_b38", "Brain_Hypothalamus"].round(5) == 0.43138
# )
# assert pd.isnull(_gene_model.loc["chr20_50957480_C_T_b38", "Brain_Hypothalamus"])

# assert pd.isnull(_gene_model.loc["chr20_50862947_C_T_b38", "Brain_Substantia_nigra"])
# assert (
#     _gene_model.loc["chr20_50957480_C_T_b38", "Brain_Substantia_nigra"].round(5)
#     == -0.1468
# )

In [127]:
gene_models = {}

for gene_id in spredixcan_genes_models.index:
    gene_models[gene_id] = _summarize_gene_models(gene_id)

In [128]:
# # testing
# _gene_id = "ENSG00000000419"

# _gene_model = gene_models[_gene_id]
# assert (
#     _gene_model.loc["chr20_50862947_C_T_b38", "Brain_Hypothalamus"].round(5) == 0.43138
# )
# assert pd.isnull(_gene_model.loc["chr20_50957480_C_T_b38", "Brain_Hypothalamus"])

# assert pd.isnull(_gene_model.loc["chr20_50862947_C_T_b38", "Brain_Substantia_nigra"])
# assert (
#     _gene_model.loc["chr20_50957480_C_T_b38", "Brain_Substantia_nigra"].round(5)
#     == -0.1468
# )

In [129]:
# save
import gzip

with gzip.GzipFile(OUTPUT_DIR_BASE / "gene_tissues_models.pkl.gz", "w") as f:
    pickle.dump(gene_models, f)

In [130]:
# testing saved file
with gzip.GzipFile(OUTPUT_DIR_BASE / "gene_tissues_models.pkl.gz", "r") as f:
    _tmp = pickle.load(f)

In [131]:
assert len(gene_models) == len(_tmp)
assert gene_models["ENSG00000000419"].equals(_tmp["ENSG00000000419"])

### Count number of _unique_ SNPs predictors used and available across tissue models

In [132]:
def _count_unique_snps(gene_id):
    """
    For a gene_id, it counts unique SNPs in all models and their intersection with GWAS SNPs (therefore, used by S-PrediXcan).
    """
    gene_tissues = spredixcan_genes_models.loc[gene_id, "tissue"]

    gene_unique_snps = set()
    for t in gene_tissues:
        t_snps = set(gene_models[gene_id].index)
        gene_unique_snps.update(t_snps)

    gene_unique_snps_in_gwas = gwas_variants_ids_set.intersection(gene_unique_snps)

    return pd.Series(
        {
            "unique_n_snps_in_model": len(gene_unique_snps),
            "unique_n_snps_used": len(gene_unique_snps_in_gwas),
        }
    )

In [133]:
# testing
spredixcan_genes_models[spredixcan_genes_models["n_snps_used_sum"] == 2].head()

,gene_name,tissue,n_tissues,tissues_pc_variances,tissues_pc_variances_cov,tissues_variances,n_snps_used_sum,n_snps_in_model_sum
gene_id,,,,,,,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra)",2,"[1.0442398458252409, 0.9557601541747592]","[0.013163142634155849, 0.00048574890347684763]","{'Brain_Hypothalamus': 0.013162153504206677, '...",2,2
ENSG00000010256,UQCRC1,"(Whole_Blood, Thyroid)",2,"[1.1048350039574655, 0.8951649960425345]","[0.0008061424968110495, 0.00023087097882836392]","{'Whole_Blood': 0.00023448891035317016, 'Thyro...",2,2
ENSG00000017427,IGF1,"(Brain_Amygdala, Testis)",2,"[1.0389319526404415, 0.9610680473595585]","[0.02060052225760429, 0.003854854256224178]","{'Brain_Amygdala': 0.02059332044751194, 'Testi...",2,2
ENSG00000043093,DCUN1D1,"(Esophagus_Muscularis, Esophagus_Gastroesophag...",2,"[1.4951653523020276, 0.5048346476979725]","[0.0005196522562331344, 0.00014798630499321565]",{'Esophagus_Muscularis': 0.0004315621222418269...,2,2
ENSG00000081377,CDC14B,"(Muscle_Skeletal, Brain_Nucleus_accumbens_basa...",2,"[1.1285007973441137, 0.8714992026558863]","[0.014647518608411531, 0.007422500206765484]","{'Muscle_Skeletal': 0.0076846621297156385, 'Br...",2,2


In [134]:
# case with two snps, not repeated across tissues
_gene_id = "ENSG00000000419"
display(
    spredixcan_gene_obj[_gene_id].get_prediction_weights(
        tissue="Brain_Hypothalamus", model_type=EQTL_MODEL
    )
)
display(
    spredixcan_gene_obj[_gene_id].get_prediction_weights(
        tissue="Brain_Substantia_nigra", model_type=EQTL_MODEL
    )
)

varID
chr20_50862947_C_T_b38    0.431375
Name: weight, dtype: float64

varID
chr20_50957480_C_T_b38   -0.146796
Name: weight, dtype: float64

In [135]:
# _tmp = _count_unique_snps(_gene_id)
# assert _tmp.shape[0] == 2
# assert _tmp["unique_n_snps_in_model"] == 2
# assert _tmp["unique_n_snps_used"] == 2

In [136]:
# get unique snps for all genes
spredixcan_genes_unique_n_snps = spredixcan_genes_models.groupby("gene_id").apply(
    lambda x: _count_unique_snps(x.name)
)

In [137]:
spredixcan_genes_unique_n_snps.head()

,unique_n_snps_in_model,unique_n_snps_used
gene_id,,
ENSG00000000419,2,2
ENSG00000000938,5,5
ENSG00000000971,12,12
ENSG00000001084,23,23
ENSG00000001167,14,13


In [138]:
assert (
    spredixcan_genes_unique_n_snps["unique_n_snps_in_model"]
    >= spredixcan_genes_unique_n_snps["unique_n_snps_used"]
).all()

In [139]:
# add unique snps to spredixcan_genes_models
spredixcan_genes_models = spredixcan_genes_models.join(spredixcan_genes_unique_n_snps)

In [140]:
spredixcan_genes_models.shape

(6444, 10)

In [141]:
spredixcan_genes_models.head()

,gene_name,tissue,n_tissues,tissues_pc_variances,tissues_pc_variances_cov,tissues_variances,n_snps_used_sum,n_snps_in_model_sum,unique_n_snps_in_model,unique_n_snps_used
gene_id,,,,,,,,,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra)",2,"[1.0442398458252409, 0.9557601541747592]","[0.013163142634155849, 0.00048574890347684763]","{'Brain_Hypothalamus': 0.013162153504206677, '...",2,2,2,2
ENSG00000000938,FGR,"(Esophagus_Muscularis, Thyroid, Brain_Cortex, ...",36,"[31.63266933637129, 2.078367751851144, 1.27161...",[0.1715948657078027],"{'Esophagus_Muscularis': 0.006536602044478473,...",40,40,5,5
ENSG00000000971,CFH,"(Esophagus_Muscularis, Thyroid, Brain_Cortex, ...",34,"[21.56076992568329, 7.310170987045463, 1.83778...","[0.2093060319587451, 0.058234347656008024, 0.0...","{'Esophagus_Muscularis': 0.01739410990047228, ...",44,44,12,12
ENSG00000001084,GCLC,"(Thyroid, Brain_Cortex, Heart_Left_Ventricle, ...",32,"[21.291939080546538, 4.699418748743195, 2.2489...","[0.39430527375123775, 0.11627488888534836, 0.0...","{'Thyroid': 0.009654918580686024, 'Brain_Corte...",46,46,23,23
ENSG00000001167,NFYA,"(Esophagus_Muscularis, Thyroid, Brain_Cortex, ...",40,"[37.3356602954977, 1.3914019604538357]","[1.1642639979821048, 0.05742881287671712]","{'Esophagus_Muscularis': 0.0318781600881541, '...",47,48,14,13


### Save

In [142]:
# this is important, other scripts depend on gene_name to be unique
assert spredixcan_genes_models["gene_name"].is_unique

In [143]:
assert not spredixcan_genes_models.isna().any(None)

In [144]:
spredixcan_genes_models.to_pickle(OUTPUT_DIR_BASE / "gene_tissues.pkl")